# Neural network playground

Go to [https://playground.tensorflow.org](https://playground.tensorflow.org) and adjust the parameters of the model there to answer the following questions.

1) With a single-hidden-layer neural network, what is the minimum number of hidden layer neurons necessary to faithfully classify the circle-surrounded-by-ring pattern?  Why is that?

### A:
Generally, at least three neurons might be needed: one to model the inner circle and two to outline the outer ring's boundaries. This setup allows the network to approximate the non-linear decision boundary between the circle and the ring. However, the exact number can vary based on the specific requirements for classification accuracy and the data's characteristics. Determining the optimal number of neurons often requires empirical testing and adjustment.

**TODO**

2) On the same dataset, describe how learning progresses with a learning rate of 3, compared to the previous default learning rate of 0.03.

**TODO**

3) Experimenting with more layers and the ReLU activation function, see whether you can train a network to classify the jelly-roll pattern (lower right) mostly correctly.

**TODO**

# Intro to Keras

Observe how to do a minimal neural network in Keras on the digits dataset.  (Note that these images are smaller than what you'd typically tackle in Keras.)
Then try to follow the same steps to build a single-layer network that classifies the wine dataset.

In [1]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers
from sklearn import datasets
digits = datasets.load_digits()
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(data, digits.target)

It's generally good to normalize data, or make the values fall between 0 and 1, when dealing with neural networks.  We got lucky in not using this step with the digits learning early in this recitation.  255 is the maximum value of brightness in the data, so that's what we divide by.

In [2]:
X_train /= 255
X_valid /= 255

Keras wants the output to be in a vector form with a 1 where the answer is and 0's elsewhere; for example, 2 becomes [0,0,1,0,0,0,0,0,0,0].  This is handled by the to_categorical() function.

In [3]:
n_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, n_classes)

Keras requires declaring the kind of model, which is normally Sequential for basic classification tasks, followed by calling .add() on the model to add layers one at a time.   There are two layers of connections in a single-hidden-layer network:  input to hidden and hidden to output.  "Dense" is a layer type that is fully connected.  Its first argument is how many neurons are in the next layer.  We put 100 for the hidden layer to make it match the scikit-learn default, then 10 because there are 10 possible outputs (the digits 0-9).  input_shape is (64,) because the digits images are 8x8 and we flattened them into 64-element vectors.  A 'softmax' activation function is like a sigmoid activation function, but normalized so the outputs all sum to 1.

In [4]:
model = Sequential()
model.add(Dense(100, activation='sigmoid', input_shape=(64,)))
model.add(Dense(10, activation='softmax'))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



Once the architecture is specified, we need to compile the model with information about what the loss is and how we will optimize to reduce that loss.  We'll choose mean squared error for the loss, and choose the somewhat fancy "adam" optimizer to minimize that loss.

In [5]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Finally, we fit to data.  Here we specify a batch size, or the number of images to look at before making a single change to the model; and the number of epochs, which should be increased if it looks as though the model might still be improving when it finishes.  Verbose=1 lets us follow along with the learning.

In [6]:
model.fit(X_train, y_train, batch_size=128,
          epochs=200, verbose=1, validation_data=(X_valid, y_valid))

Epoch 1/200


2024-02-22 11:24:25.566575: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


11/11 [==============================] - 2s 33ms/step - loss: 0.0910 - accuracy: 0.0987 - val_loss: 0.0901 - val_accuracy: 0.1044
Epoch 2/200
11/11 [==============================] - 0s 9ms/step - loss: 0.0900 - accuracy: 0.1084 - val_loss: 0.0901 - val_accuracy: 0.0867
Epoch 3/200
11/11 [==============================] - 0s 9ms/step - loss: 0.0899 - accuracy: 0.1670 - val_loss: 0.0899 - val_accuracy: 0.1267
Epoch 4/200
11/11 [==============================] - 0s 9ms/step - loss: 0.0898 - accuracy: 0.1566 - val_loss: 0.0898 - val_accuracy: 0.1022
Epoch 5/200
11/11 [==============================] - 0s 11ms/step - loss: 0.0897 - accuracy: 0.1136 - val_loss: 0.0898 - val_accuracy: 0.2667
Epoch 6/200
11/11 [==============================] - 0s 9ms/step - loss: 0.0896 - accuracy: 0.2539 - val_loss: 0.0897 - val_accuracy: 0.1800
Epoch 7/200
11/11 [==============================] - 0s 9ms/step - loss: 0.0896 - accuracy: 0.1166 - val_loss: 0.0896 - val_accuracy: 0.1533
Epoch 8/200
11/11 [====

Okay, ready to do that again?  The basic loading, splitting, and normalization of the data are done for you below.  Try to adapt the pipeline you just saw to the wine data.

In [7]:
wine = datasets.load_wine()

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(wine.data, wine.target)

In [8]:
num_examples, num_features = X_train.shape
for j in range(num_features):
  X_train[:,j] /= X_train[:,j].max()
  X_valid[:,j] /= X_valid[:,j].max()

X_train[0,:]

array([0.95886716, 0.29310345, 0.71428571, 0.54333333, 0.72839506,
       0.90909091, 0.76335878, 0.39393939, 0.56703911, 0.49076923,
       0.5497076 , 0.8275    , 0.57738095])

In [9]:
# TODO to_categorical
n_classes = len(set(y_train))  # Determine the number of unique classes
y_train = tf.keras.utils.to_categorical(y_train, n_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, n_classes)

In [10]:
# TODO create model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(100, activation='sigmoid', input_shape=(13,)))  # Wine dataset has 13 features
model.add(Dense(n_classes, activation='softmax'))  # Output layer for n_classes categories


In [11]:
# TODO compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


In [12]:
# TODO fit model
model.fit(X_train, y_train, batch_size=128, epochs=200, verbose=1, validation_data=(X_valid, y_valid))


Epoch 1/200
2/2 [==============================] - 1s 377ms/step - loss: 0.2240 - accuracy: 0.3910 - val_loss: 0.2206 - val_accuracy: 0.4222
Epoch 2/200
2/2 [==============================] - 0s 27ms/step - loss: 0.2216 - accuracy: 0.3910 - val_loss: 0.2199 - val_accuracy: 0.4222
Epoch 3/200
2/2 [==============================] - 0s 29ms/step - loss: 0.2212 - accuracy: 0.3910 - val_loss: 0.2193 - val_accuracy: 0.4222
Epoch 4/200
2/2 [==============================] - 0s 26ms/step - loss: 0.2209 - accuracy: 0.3910 - val_loss: 0.2189 - val_accuracy: 0.4222
Epoch 5/200
2/2 [==============================] - 0s 26ms/step - loss: 0.2202 - accuracy: 0.3910 - val_loss: 0.2187 - val_accuracy: 0.4222
Epoch 6/200
2/2 [==============================] - 0s 27ms/step - loss: 0.2194 - accuracy: 0.3910 - val_loss: 0.2187 - val_accuracy: 0.4222
Epoch 7/200
2/2 [==============================] - 0s 26ms/step - loss: 0.2188 - accuracy: 0.3910 - val_loss: 0.2185 - val_accuracy: 0.4222
Epoch 8/200
2/2 [==

# Deeper neural networks

Now, we'll try to use a deep neural network to classify the small images in the dataset CIFAR10.  These are small (32x32) color images of 10 categories of things, including cats and airplanes.  You can browse the dataset at: https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=cifar10

In [13]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 9s 0us/step


As a baseline, try again fitting the MNIST network at https://keras.io/examples/vision/mnist_convnet/ to this data.  You'll probably benefit from more epochs - try around 50.

In [14]:
X_train = X_train/255
X_test = X_test/255

In [16]:
# TODO to_categorical, create model, compile model, fit model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (for use with categorical_crossentropy)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, kernel_size=(3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(10, activation="softmax"),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=50, validation_data=(x_test, y_test))


Epoch 1/50
391/391 [==============================] - 10s 22ms/step - loss: 1.6900 - accuracy: 0.3786 - val_loss: 1.3918 - val_accuracy: 0.5007
Epoch 2/50
391/391 [==============================] - 8s 20ms/step - loss: 1.3677 - accuracy: 0.5107 - val_loss: 1.1906 - val_accuracy: 0.5739
Epoch 3/50
391/391 [==============================] - 8s 20ms/step - loss: 1.2290 - accuracy: 0.5616 - val_loss: 1.1203 - val_accuracy: 0.6051
Epoch 4/50
391/391 [==============================] - 8s 20ms/step - loss: 1.1366 - accuracy: 0.5978 - val_loss: 1.0213 - val_accuracy: 0.6465
Epoch 5/50
391/391 [==============================] - 8s 20ms/step - loss: 1.0670 - accuracy: 0.6227 - val_loss: 0.9872 - val_accuracy: 0.6555
Epoch 6/50
391/391 [==============================] - 8s 21ms/step - loss: 1.0151 - accuracy: 0.6441 - val_loss: 0.9365 - val_accuracy: 0.6741
Epoch 7/50
391/391 [==============================] - 8s 20ms/step - loss: 0.9706 - accuracy: 0.6600 - val_loss: 0.9560 - val_accuracy: 0.658

Now try to improve the performance by adding another couple of layers, following the pattern of adding successive Conv2D and MaxPooling2D blocks that double the number of filters each time (32, 64, 128...).

In [17]:
# TODO as above but experiment with extra layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),  # Additional Conv2D layer
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=128, epochs=50, validation_data=(x_test, y_test))

Epoch 1/50
391/391 [==============================] - 13s 31ms/step - loss: 1.7632 - accuracy: 0.3451 - val_loss: 1.4100 - val_accuracy: 0.4821
Epoch 2/50
391/391 [==============================] - 9s 23ms/step - loss: 1.4090 - accuracy: 0.4890 - val_loss: 1.2513 - val_accuracy: 0.5516
Epoch 3/50
391/391 [==============================] - 8s 21ms/step - loss: 1.2481 - accuracy: 0.5554 - val_loss: 1.1453 - val_accuracy: 0.5937
Epoch 4/50
391/391 [==============================] - 8s 21ms/step - loss: 1.1332 - accuracy: 0.6024 - val_loss: 1.0484 - val_accuracy: 0.6337
Epoch 5/50
391/391 [==============================] - 8s 21ms/step - loss: 1.0444 - accuracy: 0.6347 - val_loss: 0.9629 - val_accuracy: 0.6587
Epoch 6/50
391/391 [==============================] - 9s 22ms/step - loss: 0.9634 - accuracy: 0.6629 - val_loss: 0.9069 - val_accuracy: 0.6829
Epoch 7/50
391/391 [==============================] - 9s 22ms/step - loss: 0.9045 - accuracy: 0.6848 - val_loss: 0.9216 - val_accuracy: 0.680